<a href="https://colab.research.google.com/github/Deepakpanchal24/Class_project/blob/main/NLP%2BDL_Pragatee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas numpy tensorflow transformers scikit-learn gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr

# Load dataset
df = pd.read_csv("/content/synthetic_medical_data_genai.csv")
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "")

# Drop invalid rows
df = df.dropna(subset=["symptoms", "differential_diagnoses"])

# Combine input fields for embedding
df["combined_input"] = (
    df["symptoms"].fillna('') + " " +
    df["medical_history"].fillna('') + " " +
    df["test_results"].fillna('')
)

# Load SentenceTransformer model
model = SentenceTransformer("all-MiniLM-L6-v2")
case_embeddings = model.encode(df["combined_input"].tolist(), convert_to_tensor=True, device='cpu')

# Main prediction function with filters
def get_diagnosis_filtered(symptoms, history, test_results, age, gender, region):
    # Filter dataset by age, gender, region (simple logic)
    df_filtered = df.copy()

    # Apply filters if provided
    if age:
        try:
            age = int(age)
            df_filtered = df_filtered[(df_filtered['age'].astype(int) >= age - 5) & (df_filtered['age'].astype(int) <= age + 5)]
        except:
            pass  # If user enters non-numeric, skip

    if gender:
        gender = gender.strip().lower()
        df_filtered = df_filtered[df_filtered['gender'].str.strip().str.lower() == gender]

    if region:
        region = region.strip().lower()
        df_filtered = df_filtered[df_filtered['region'].str.strip().str.lower() == region]

    if df_filtered.empty:
        return "❌ No matching cases found for the given filters. Please broaden your filters."

    # Encode filtered cases
    filtered_embeddings = model.encode(df_filtered["combined_input"].tolist(), convert_to_tensor=True, device='cpu')

    # Encode user input
    user_input = symptoms + " " + history + " " + test_results
    user_embedding = model.encode([user_input], convert_to_tensor=True, device='cpu')

    # Compute similarity
    similarity_scores = cosine_similarity(user_embedding.cpu(), filtered_embeddings.cpu())[0]
    best_idx = similarity_scores.argmax()
    match = df_filtered.iloc[best_idx]

    # Final output
    result = f"""
### ✅ Differential Diagnoses
- {match['differential_diagnoses']}

### 💊 Medications
- {match['medications']}

### 🩺 Treatment Plan
- {match['treatment_plan']}

### 🔁 Follow Ups
- {match['follow_ups']}
"""
    return result

# Build Gradio UI
demo = gr.Interface(
    fn=get_diagnosis_filtered,
    inputs=[
        gr.Textbox(label="Symptoms"),
        gr.Textbox(label="Medical History"),
        gr.Textbox(label="Test Results"),
        gr.Textbox(label="Age (Optional)", placeholder="e.g. 45"),
        gr.Textbox(label="Gender (Optional)", placeholder="male / female"),
        gr.Textbox(label="Region (Optional)", placeholder="e.g. rural / urban")
    ],
    outputs=gr.Markdown(label="Diagnosis & Treatment"),
    title="🧠 MediMind Diagnostic Support (DL + NLP + Filter)",
    description="AI assistant with age/gender/region filters for better personalized results."
)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://98cf4bd3593caad742.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
